### load the model and realtime analysis

In [7]:
import cv2
import numpy as np
import time
import dlib
from scipy.spatial import distance as dist
import tflite_runtime.interpreter as tflite
import tensorflow as tf

# Function to calculate eye aspect ratio (EAR)
def calculate_ear(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

# Function to calculate mouth aspect ratio (MAR)
def calculate_mar(mouth):
    A = dist.euclidean(mouth[2], mouth[10])
    B = dist.euclidean(mouth[4], mouth[8])
    C = dist.euclidean(mouth[0], mouth[6])
    return (A + B) / (2.0 * C)

# Load dlib's face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # Pre-trained landmark model

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='drowsiness_model.tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Real-time detection
cap = cv2.VideoCapture(1)

drowsy_start_time = None  # Initialize the drowsy start time
drowsy_threshold = 4      # Threshold for drowsiness in seconds

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    is_drowsy = False  # Track if the current frame detects drowsiness

    for face in faces:
        # Detect landmarks
        landmarks = predictor(gray, face)
        left_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)])
        right_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)])
        mouth = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(48, 68)])

        # Calculate EAR and MAR
        ear = (calculate_ear(left_eye) + calculate_ear(right_eye)) / 2.0
        mar = calculate_mar(mouth)

        # Prepare input for the model
        input_data = np.array([[ear, mar]], dtype=np.float32)
        interpreter.set_tensor(input_details[0]['index'], input_data)

        # Perform inference
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        prediction = int(output_data[0] > 0.99)  # 1 for drowsy, 0 for awake

        if prediction == 1:
            is_drowsy = True  # Set flag for this frame
            color = (0, 0, 255)  # Red for drowsy
        else:
            color = (0, 255, 0)  # Green for awake

        # Draw rectangle around the face
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    # Track continuous drowsiness
    if is_drowsy:
        if drowsy_start_time is None:
            drowsy_start_time = time.time()  # Start timer
        elif time.time() - drowsy_start_time >= drowsy_threshold:
            cv2.putText(frame, "DROWSY!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
    else:
        drowsy_start_time = None  # Reset timer if not drowsy

    # Display the frame
    cv2.imshow("Drowsiness Detection", frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


/var/folders/5d/cxg576h92j906wv9p0xyjc3c0000gn/T/ipykernel_91219/1047305117.py:69: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prediction = int(output_data[0] > 0.3)  # 1 for drowsy, 0 for awake


: 